# Combination Generator

In [182]:
import pandas as pd
import numpy as np
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.21.1")
db = lusas.database()

Read the excel definitions

In [183]:
file_name = "12 Combinations.xlsx"
actions_sheet = "Actions"
do_print = False

In [184]:
df_permanent = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(0,3), header=1).dropna()
if do_print : print(df_permanent.head())

In [185]:
df_variable = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(4,9), header=1).dropna()
if do_print : print(df_variable.head())

In [186]:
df_prestress = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(10,13), header=1).dropna()
if do_print : print(df_prestress.head())

In [187]:
df_accidental = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(14,16), header=1).dropna()
if do_print : print(df_accidental.head())

In [188]:
df_perm_loadcases = pd.read_excel(file_name, sheet_name="PermanentLoadcases")
print(df_perm_loadcases.head())
dict_perm_loadcases = df_perm_loadcases.set_index('Action Name').T.to_dict('list')
print(dict_perm_loadcases)

      Action Name        Loadcase Name      Loadcase Name.1
0  Permanent load          Self Weight  Self Weight Railing
1    SuperImposed            Surfacing                  NaN
2      Settlement  Settlement Envelope                  NaN
3       Shrinkage            Shrinkage                  NaN
{'Permanent load': ['Self Weight', 'Self Weight Railing'], 'SuperImposed': ['Surfacing', nan], 'Settlement': ['Settlement Envelope', nan], 'Shrinkage': ['Shrinkage', nan]}


In [189]:
df_var_loadcases = pd.read_excel(file_name, sheet_name="VariableLoadcases")
if do_print : print(df_var_loadcases.head())
dict_var_loadcases = df_var_loadcases.set_index('Action Name').T.to_dict('list')

In [190]:
df_acc_loadcases = pd.read_excel(file_name, sheet_name="AccidentalLoadcases")
if do_print : print(df_acc_loadcases.head())

In [191]:
for i, row in df_variable.iterrows():
    action = row["Var Action Name"]
    load_factor = row["DesignFactor"]
    comb_factor = row["CombinationFactor"]
    freq_factor = row["FrequentFactor"]
    quas_factor = row["QuasiFactor"]
    
    smart_comb = db.createCombinationSmart(f"ULS - {action}")


    # Permanent actions
    for action_type in dict_perm_loadcases.keys():
        # Factors for this type
        beneficial = df_permanent[df_permanent['Perm Action Name'] == action_type]['Beneficial']
        adverse    = df_permanent[df_permanent['Perm Action Name'] == action_type]['Adverse']

        # LUSAS Loadcases
        for loadcase in dict_perm_loadcases[action_type]:
            if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

            if db.existsLoadset(loadcase):
                loadset = db.getLoadset(loadcase)
                type_code = loadset.getTypeCode()

                if type_code <= 2:
                    smart_comb.addEntry(beneficial, (adverse-beneficial), loadset)
                elif type_code == 3:
                    envelope1 = win32.CastTo(loadset, "IFEnvelope")
                    envelope2 = envelope1.getAssocLoadset()
                    smart_comb.addEntry(beneficial, (adverse-beneficial), envelope1)
                    smart_comb.addEntry(beneficial, (adverse-beneficial), envelope2)
                elif type_code == 6:
                    smart1 = win32.CastTo(loadset, "IFSmartCombination")
                    smart2 = smart1.getAssocLoadset()
                    smart_comb.addEntry(beneficial, (adverse-beneficial), smart1)
                    smart_comb.addEntry(beneficial, (adverse-beneficial), smart2)
            else:
                print(f"Permanent loadcase '{loadcase}' does not exists")


    # Variable actions
    for action_type in dict_var_loadcases.keys():
        # Determine variable factor
        if action_type == action:
            adverse_factor = load_factor                        # Leading Variable
        else:
            adverse_factor = load_factor * comb_factor          # Accompanying Variable

        # LUSAS Loadcases
        for loadcase in dict_var_loadcases[action_type]:
            if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

            if db.existsLoadset(loadcase):
                loadset = db.getLoadset(loadcase)
                type_code = loadset.getTypeCode()

                if type_code <= 2:
                    smart_comb.addEntry(0.0, adverse_factor, loadset)
                elif type_code == 3:
                    envelope1 = win32.CastTo(loadset, "IFEnvelope")
                    envelope2 = envelope1.getAssocLoadset()
                    smart_comb.addEntry(0.0, adverse_factor, envelope1)
                    smart_comb.addEntry(0.0, adverse_factor, envelope2)
                elif type_code == 6:
                    smart1 = win32.CastTo(loadset, "IFSmartCombination")
                    smart2 = smart1.getAssocLoadset()
                    smart_comb.addEntry(0.0, adverse_factor, smart1)
                    smart_comb.addEntry(0.0, adverse_factor, smart2)
            else:
                print(f"Variable loadcase '{loadcase}' does not exists")






C:\Users\rob\AppData\Local\Temp\gen_py\3.12\AC0CF861-2101-11CF-91D8-0020AFC19F64x0x21x1\IFSmartCombination.py:46: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ret = self._oleobj_.InvokeTypes(1610874881, LCID, 1, (9, 0), ((5, 1), (5, 1), (12, 1), (12, 17), (12, 17), (12, 17)),factor
